# Downloading the data

This notebook is used to download the ERA5 data from weatherbench2.


In [7]:
import xarray as xr
import os
import fsspec
import numpy as np
import pandas as pd
import jinja2

In [ ]:
#!gcloud auth login
#!gsutil -m cp -r "gs://weatherbench2/datasets/era5/1959-2022-6h-64x32_equiangular_conservative.zarr" ./ERA5_data
#!gsutil -m cp -r "gs://weatherbench2/datasets/era5-hourly-climatology/1990-2019_6h_64x32_equiangular_conservative.zarr" ./ERA5_data

In [8]:

# Define the relative path to the file
relative_path = "ERA5_data/1959-2022-6h-64x32_equiangular_conservative.zarr"

# Get the absolute path by joining the current directory with the relative path
absolute_path = os.path.join(os.path.dirname(os.getcwd()), relative_path)
print(absolute_path)


/home/jupyter-lukas/Masters-Thesis/ERA5_data/1959-2022-6h-64x32_equiangular_conservative.zarr


In [9]:
# Open the Zarr file using xarray
obs_data = xr.open_zarr(absolute_path)
obs_data


<xarray.Dataset>
Dimensions:                                           (time: 92044,
                                                       longitude: 64,
                                                       latitude: 32, level: 13)
Coordinates:
  * latitude                                          (latitude) float64 -87....
  * level                                             (level) int64 50 ... 1000
  * longitude                                         (longitude) float64 0.0...
  * time                                              (time) datetime64[ns] 1...
Data variables: (12/38)
    10m_u_component_of_wind                           (time, longitude, latitude) float32 dask.array<chunksize=(100, 64, 32), meta=np.ndarray>
    10m_v_component_of_wind                           (time, longitude, latitude) float32 dask.array<chunksize=(100, 64, 32), meta=np.ndarray>
    10m_wind_speed                                    (time, longitude, latitude) float32 dask.array<chunksize=(100, 64, 32), meta=np.ndarray>
    2m_temperature                                    (time, longitude, latitude) float32 dask.array<chunksize=(100, 64, 32), meta=np.ndarray>
    angle_of_sub_gridscale_orography                  (longitude, latitude) float32 dask.array<chunksize=(64, 32), meta=np.ndarray>
    anisotropy_of_sub_gridscale_orography             (longitude, latitude) float32 dask.array<chunksize=(64, 32), meta=np.ndarray>
    ...                                                ...
    type_of_high_vegetation                           (longitude, latitude) float32 dask.array<chunksize=(64, 32), meta=np.ndarray>
    type_of_low_vegetation                            (longitude, latitude) float32 dask.array<chunksize=(64, 32), meta=np.ndarray>
    u_component_of_wind                               (time, level, longitude, latitude) float32 dask.array<chunksize=(100, 13, 64, 32), meta=np.ndarray>
    v_component_of_wind                               (time, level, longitude, latitude) float32 dask.array<chunksize=(100, 13, 64, 32), meta=np.ndarray>
    vertical_velocity                                 (time, level, longitude, latitude) float32 dask.array<chunksize=(100, 13, 64, 32), meta=np.ndarray>
    wind_speed                                        (time, level, longitude, latitude) float32 dask.array<chunksize=(100, 13, 64, 32), meta=np.ndarray>

In [10]:
# prune the dataset of unnecessary variables.
for x in obs_data.data_vars:
    if x not in ['2m_temperature','mean_sea_level_pressure','10m_v_component_of_wind',
        '10m_u_component_of_wind','total_precipitation_6hr','toa_incident_solar_radiation',
        'temperature','geopotential','geopotential_at_surface','land_sea_mask',
        'u_component_of_wind','v_component_of_wind','vertical_velocity','specific_humidity']:
        obs_data = obs_data.drop_vars(x)

obs_data

<xarray.Dataset>
Dimensions:                       (time: 92044, longitude: 64, latitude: 32,
                                   level: 13)
Coordinates:
  * latitude                      (latitude) float64 -87.19 -81.56 ... 87.19
  * level                         (level) int64 50 100 150 200 ... 850 925 1000
  * longitude                     (longitude) float64 0.0 5.625 ... 348.8 354.4
  * time                          (time) datetime64[ns] 1959-01-01 ... 2021-1...
Data variables: (12/14)
    10m_u_component_of_wind       (time, longitude, latitude) float32 dask.array<chunksize=(100, 64, 32), meta=np.ndarray>
    10m_v_component_of_wind       (time, longitude, latitude) float32 dask.array<chunksize=(100, 64, 32), meta=np.ndarray>
    2m_temperature                (time, longitude, latitude) float32 dask.array<chunksize=(100, 64, 32), meta=np.ndarray>
    geopotential                  (time, level, longitude, latitude) float32 dask.array<chunksize=(100, 13, 64, 32), meta=np.ndarray>
    geopotential_at_surface       (longitude, latitude) float32 dask.array<chunksize=(64, 32), meta=np.ndarray>
    land_sea_mask                 (longitude, latitude) float32 dask.array<chunksize=(64, 32), meta=np.ndarray>
    ...                            ...
    temperature                   (time, level, longitude, latitude) float32 dask.array<chunksize=(100, 13, 64, 32), meta=np.ndarray>
    toa_incident_solar_radiation  (time, longitude, latitude) float32 dask.array<chunksize=(100, 64, 32), meta=np.ndarray>
    total_precipitation_6hr       (time, longitude, latitude) float32 dask.array<chunksize=(100, 64, 32), meta=np.ndarray>
    u_component_of_wind           (time, level, longitude, latitude) float32 dask.array<chunksize=(100, 13, 64, 32), meta=np.ndarray>
    v_component_of_wind           (time, level, longitude, latitude) float32 dask.array<chunksize=(100, 13, 64, 32), meta=np.ndarray>
    vertical_velocity             (time, level, longitude, latitude) float32 dask.array<chunksize=(100, 13, 64, 32), meta=np.ndarray>

In [11]:
#rename obs data:
obs_data = obs_data.rename({'longitude': 'lon'})
obs_data = obs_data.rename({'latitude': 'lat'})

# add datetime coord:
copied_coord = obs_data['time'].copy()
# Rename the copied coordinate to the new name
obs_data = obs_data.assign_coords(datetime=copied_coord)

# add batch dim:
obs_data = obs_data.expand_dims('batch')
obs_data['datetime'] = obs_data['datetime'].expand_dims('batch')

obs_data

<xarray.Dataset>
Dimensions:                       (batch: 1, time: 92044, lon: 64, lat: 32,
                                   level: 13)
Coordinates:
  * lat                           (lat) float64 -87.19 -81.56 ... 81.56 87.19
  * level                         (level) int64 50 100 150 200 ... 850 925 1000
  * lon                           (lon) float64 0.0 5.625 11.25 ... 348.8 354.4
  * time                          (time) datetime64[ns] 1959-01-01 ... 2021-1...
    datetime                      (batch, time) datetime64[ns] 1959-01-01 ......
Dimensions without coordinates: batch
Data variables: (12/14)
    10m_u_component_of_wind       (batch, time, lon, lat) float32 dask.array<chunksize=(1, 100, 64, 32), meta=np.ndarray>
    10m_v_component_of_wind       (batch, time, lon, lat) float32 dask.array<chunksize=(1, 100, 64, 32), meta=np.ndarray>
    2m_temperature                (batch, time, lon, lat) float32 dask.array<chunksize=(1, 100, 64, 32), meta=np.ndarray>
    geopotential                  (batch, time, level, lon, lat) float32 dask.array<chunksize=(1, 100, 13, 64, 32), meta=np.ndarray>
    geopotential_at_surface       (batch, lon, lat) float32 dask.array<chunksize=(1, 64, 32), meta=np.ndarray>
    land_sea_mask                 (batch, lon, lat) float32 dask.array<chunksize=(1, 64, 32), meta=np.ndarray>
    ...                            ...
    temperature                   (batch, time, level, lon, lat) float32 dask.array<chunksize=(1, 100, 13, 64, 32), meta=np.ndarray>
    toa_incident_solar_radiation  (batch, time, lon, lat) float32 dask.array<chunksize=(1, 100, 64, 32), meta=np.ndarray>
    total_precipitation_6hr       (batch, time, lon, lat) float32 dask.array<chunksize=(1, 100, 64, 32), meta=np.ndarray>
    u_component_of_wind           (batch, time, level, lon, lat) float32 dask.array<chunksize=(1, 100, 13, 64, 32), meta=np.ndarray>
    v_component_of_wind           (batch, time, level, lon, lat) float32 dask.array<chunksize=(1, 100, 13, 64, 32), meta=np.ndarray>
    vertical_velocity             (batch, time, level, lon, lat) float32 dask.array<chunksize=(1, 100, 13, 64, 32), meta=np.ndarray>

In [12]:
obs_data['2m_temperature'].values

array([[[[244.60733, 246.23764, 244.14255, ..., 262.59756, 244.4864 ,
          242.59596],
         [244.31146, 244.46599, 242.29948, ..., 268.52905, 247.15837,
          242.97812],
         [243.68727, 242.95036, 240.6064 , ..., 270.03912, 249.18608,
          243.03012],
         ...,
         [246.50488, 251.295  , 256.54962, ..., 246.04909, 242.17416,
          241.77927],
         [246.03793, 249.19165, 252.03925, ..., 247.61113, 243.85313,
          241.98943],
         [245.12758, 247.6642 , 246.26146, ..., 256.0192 , 244.7451 ,
          242.29877]],

        [[244.94489, 246.7836 , 245.48953, ..., 262.8359 , 244.44336,
          242.26163],
         [244.39612, 245.28886, 243.97768, ..., 268.9111 , 247.22038,
          242.66519],
         [243.77327, 244.15395, 242.80005, ..., 270.2984 , 248.8203 ,
          242.68434],
         ...,
         [246.83675, 251.78487, 257.27344, ..., 245.82948, 241.78658,
          241.3578 ],
         [245.93636, 249.39265, 253.13072, ..., 24

In [13]:
# save obs_data
obs_data.to_zarr('/home/jupyter-lukas/Masters-Thesis/ERA5_data/obs_data.zarr', mode='w')

In [14]:
obs_data = xr.open_zarr('/home/jupyter-lukas/Masters-Thesis/ERA5_data/obs_data.zarr')
obs_data

<xarray.Dataset>
Dimensions:                       (batch: 1, time: 92044, lon: 64, lat: 32,
                                   level: 13)
Coordinates:
    datetime                      (batch, time) datetime64[ns] dask.array<chunksize=(1, 23011), meta=np.ndarray>
  * lat                           (lat) float64 -87.19 -81.56 ... 81.56 87.19
  * level                         (level) int64 50 100 150 200 ... 850 925 1000
  * lon                           (lon) float64 0.0 5.625 11.25 ... 348.8 354.4
  * time                          (time) datetime64[ns] 1959-01-01 ... 2021-1...
Dimensions without coordinates: batch
Data variables: (12/14)
    10m_u_component_of_wind       (batch, time, lon, lat) float32 dask.array<chunksize=(1, 100, 64, 32), meta=np.ndarray>
    10m_v_component_of_wind       (batch, time, lon, lat) float32 dask.array<chunksize=(1, 100, 64, 32), meta=np.ndarray>
    2m_temperature                (batch, time, lon, lat) float32 dask.array<chunksize=(1, 100, 64, 32), meta=np.ndarray>
    geopotential                  (batch, time, level, lon, lat) float32 dask.array<chunksize=(1, 100, 13, 64, 32), meta=np.ndarray>
    geopotential_at_surface       (batch, lon, lat) float32 dask.array<chunksize=(1, 64, 32), meta=np.ndarray>
    land_sea_mask                 (batch, lon, lat) float32 dask.array<chunksize=(1, 64, 32), meta=np.ndarray>
    ...                            ...
    temperature                   (batch, time, level, lon, lat) float32 dask.array<chunksize=(1, 100, 13, 64, 32), meta=np.ndarray>
    toa_incident_solar_radiation  (batch, time, lon, lat) float32 dask.array<chunksize=(1, 100, 64, 32), meta=np.ndarray>
    total_precipitation_6hr       (batch, time, lon, lat) float32 dask.array<chunksize=(1, 100, 64, 32), meta=np.ndarray>
    u_component_of_wind           (batch, time, level, lon, lat) float32 dask.array<chunksize=(1, 100, 13, 64, 32), meta=np.ndarray>
    v_component_of_wind           (batch, time, level, lon, lat) float32 dask.array<chunksize=(1, 100, 13, 64, 32), meta=np.ndarray>
    vertical_velocity             (batch, time, level, lon, lat) float32 dask.array<chunksize=(1, 100, 13, 64, 32), meta=np.ndarray>

In [15]:
obs_data['2m_temperature'].values

array([[[[244.60733, 246.23764, 244.14255, ..., 262.59756, 244.4864 ,
          242.59596],
         [244.31146, 244.46599, 242.29948, ..., 268.52905, 247.15837,
          242.97812],
         [243.68727, 242.95036, 240.6064 , ..., 270.03912, 249.18608,
          243.03012],
         ...,
         [246.50488, 251.295  , 256.54962, ..., 246.04909, 242.17416,
          241.77927],
         [246.03793, 249.19165, 252.03925, ..., 247.61113, 243.85313,
          241.98943],
         [245.12758, 247.6642 , 246.26146, ..., 256.0192 , 244.7451 ,
          242.29877]],

        [[244.94489, 246.7836 , 245.48953, ..., 262.8359 , 244.44336,
          242.26163],
         [244.39612, 245.28886, 243.97768, ..., 268.9111 , 247.22038,
          242.66519],
         [243.77327, 244.15395, 242.80005, ..., 270.2984 , 248.8203 ,
          242.68434],
         ...,
         [246.83675, 251.78487, 257.27344, ..., 245.82948, 241.78658,
          241.3578 ],
         [245.93636, 249.39265, 253.13072, ..., 24